In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="GqlHqtqITXwoJT8K9PtR")
project = rf.workspace("projetpfe").project("plastic-detection-blsft")
dataset = project.version(1).download("multiclass")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...

Extracting Dataset Version Zip to Plastic-detection-1 in multiclass:: 100%|██████████| 9422/9422 [00:03<00:00, 2670.25it/s]


In [ ]:
df = pd.read_csv("/content/Plastic-detection-1/train/_classes.csv", delimiter=",")
dict_plastic = {}

for number, name in enumerate(df.columns):
    if name == 'filename':continue              # skip 1st column
    dict_plastic[number-1] = name.strip()
dict_plastic = {key: dict_plastic[key] for key in dict_plastic if key != 'Unlabeled'}
img_size = 224

In [ ]:
def loadImage(path, label, image_size=(224,224), return_label=False):
    labels = pd.read_csv(label)
    labels = labels.sort_values('filename')
    labels = labels.loc[:, labels.columns != 'filename']
    labels = np.array(labels.idxmax(axis=1).str.strip().astype(
            'category').cat.codes).reshape(-1,1)
    
    if return_label:
        return labels
    images = tf.keras.utils.image_dataset_from_directory(path, shuffle=False, batch_size=None, label_mode=None, image_size=image_size)
    labels = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((images, labels))

In [ ]:
path_train = "/content/Plastic-detection-1/train/"
path_valid = "/content/Plastic-detection-1/valid/"
path_test = "/content/Plastic-detection-1/test/"
train_dataset = loadImage(path_train, path_train+"_classes.csv", image_size=(img_size,img_size))
valid_dataset = loadImage(path_valid, path_valid+"_classes.csv", image_size=(img_size,img_size))
test_dataset = loadImage(path_test, path_test+"_classes.csv", image_size=(img_size,img_size))
test_labels = loadImage(path_test, path_test+"_classes.csv", return_label=True)

Found 8110 files belonging to 1 classes.
Found 869 files belonging to 1 classes.
Found 435 files belonging to 1 classes.


In [ ]:
train = train_dataset.cache('train1').shuffle(500).batch(32)
valid = valid_dataset.cache().batch(32)
test = test_dataset.cache().batch(32)

In [ ]:
model2 = tf.keras.applications.MobileNetV3Large(
    input_shape=None,
    alpha=1.0,
    minimalistic=False,
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    classes=1000,
    pooling=None,
    dropout_rate=0.2,
    classifier_activation='softmax',
    include_preprocessing=True
    
)
model2.trainable=False

In [ ]:
model= tf.keras.models.Sequential([
    model2, 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(len(dict_plastic),"softmax")
])
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer="adam",
    metrics=['accuracy']
)
model.build([None,224,224,3])
model.summary(
)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3large (Functiona  (None, 1000)             5507432   
 l)                                                              
                                                                 
 flatten_3 (Flatten)         (None, 1000)              0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 5,508,433
Trainable params: 1,001
Non-trainable params: 5,507,432
_________________________________________________________________


In [ ]:
model.fit(
    train,
    validation_data=valid,
    epochs=10

)

Epoch 1/10


InvalidArgumentError: ignored